In [29]:
import numpy as np
import pandas as pd
from scipy import optimize
from scipy import stats
import matplotlib.pyplot as plt
import numba
from scipy import interpolate
import quantecon as qe
from IPython.core.display import HTML

# Question 5

## a) Deriving the Bellman equations

The only state variable in the model is current production. We can write the different value functions as:

\begin{equation}\begin{aligned}
Q(p) &=w(p)+\delta((1-s) E[Q(\tilde{p})]+s E[U(\tilde{p})]) \\
U(p) &=z+\delta\left(\frac{m(u, v)}{u} E[Q(\tilde{p})]+\left(1-\frac{m(u, v)}{u}\right) E[U(\tilde{p})]\right) \\
J(p) &=p-w(p)+\delta((1-s) E[J(\tilde{p})]+s E[V(\tilde{p})]) \\
V(p) &=-c(p)+\delta\left(\frac{m(u, v)}{v} E[J(\tilde{p})]+\left(1-\frac{m(u, v)}{v}\right) E[V(\tilde{p})]\right)
\end{aligned}\end{equation}


## b) Getting the transition matrix from Rouwenhorst method

In [30]:
z     = 0.4
β     = 0.4
l     = 0.407
c_k   = 0.474
c_l   = 0.11
ξ     = 0.449
δ     = 0.99 ** (1/12)
s     = 0.0081
σ     = 0.0034
ρ     = 0.9895



In [80]:
MC = qe.markov.approximation.rouwenhorst(10, 0, σ, ρ)

In [32]:
Π = MC.P
frame = pd.DataFrame(Π)
HTML(frame.round(4).to_html())

,0,1,2,3,4,5,6,7,8,9
0,0.9537,0.0453,0.0010,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1,0.0050,0.9539,0.0403,0.0007,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2,0.0000,0.0101,0.9541,0.0352,0.0006,0.0000,0.0000,0.0000,0.0000,0.0000
3,0.0000,0.0001,0.0151,0.9542,0.0302,0.0004,0.0000,0.0000,0.0000,0.0000
4,0.0000,0.0000,0.0002,0.0201,0.9543,0.0252,0.0003,0.0000,0.0000,0.0000
5,0.0000,0.0000,0.0000,0.0003,0.0252,0.9543,0.0201,0.0002,0.0000,0.0000
6,0.0000,0.0000,0.0000,0.0000,0.0004,0.0302,0.9542,0.0151,0.0001,0.0000
7,0.0000,0.0000,0.0000,0.0000,0.0000,0.0006,0.0352,0.9541,0.0101,0.0000
8,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0007,0.0403,0.9539,0.0050
9,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0010,0.0453,0.9537


This is the transition matrix of the Rouwenhorst approximation with N=10 states. $$ \Pi_{i,j} = P(p'=j | p = i)$$

In [6]:
MC.stationary_distributions

array([[0.00195312, 0.01757812, 0.0703125 , 0.1640625 , 0.24609375,
        0.24609375, 0.1640625 , 0.0703125 , 0.01757812, 0.00195312]])

In [7]:
MC.stationary_distributions @ Π

array([[0.00195312, 0.01757812, 0.0703125 , 0.1640625 , 0.24609375,
        0.24609375, 0.1640625 , 0.0703125 , 0.01757812, 0.00195312]])

In [33]:
@numba.njit
def matching(u,v):
    return u*v/((u**l + v**l)**(1/l))

@numba.njit
def costvac(P):
    return c_k * P + c_l * np.power(P,ξ)


In [34]:
Pss= np.mean(MC.simulate(ts_length=10_000_000))   # Productivity at steady state

In [35]:
print(Pss)

-7.475476085130794e-05


## c) Solving for market tightness

We want to derive a relationship between productivity and market tightness. By the free entry condition we know that $V(p) = 0$ for all values of p.

Therefore we have:

\begin{equation}\begin{aligned}
    c(p) &= \delta\left(\frac{m(1, \theta)}{\theta} E[J(\tilde{p})]\right)
\end{aligned}\end{equation}

with $ \theta = \frac{v}{u}$

From the Nash Bargaining equation we get:
\begin{aligned}
(1-\beta)(Q(p)-U(p)) &=\beta J(p) \\
\Rightarrow(1-\beta) E[Q(p)-U(p)] &=\beta E[J(p)]
\end{aligned}

From the value function of the worker:
\begin{aligned}
Q(p)-U(p) &=w(p)+\delta((1-s) E[Q(\tilde{p})]+s E[U(\tilde{p})]) \\
&-z-\delta(m(1, \theta) E[Q(\tilde{p})]-(1-m(1, \theta)) E[U(\tilde{p})]) \\
&=w(p)-z+\delta(1-s-m(1, \theta)) E[Q(\tilde{p})-U(\tilde{p})]
\end{aligned}

And therefore the surplus is:

\begin{aligned}
S(p) &=w(p)-z+\delta(1-s-m(1, \theta)) E[Q(\tilde{p})-U(\tilde{p})]+p-w(p)+\delta(1-s) E[J(\tilde{p})] \\
&=p-z+\delta(1-s) E[Q(\tilde{p})-U(\tilde{p})+J(\tilde{p})]-\delta m(1, \theta) E[Q(\tilde{p})-U(\tilde{p})] \\
&=p-z+\delta(1-s) E[S(\tilde{p})]-\delta m(1, \theta) \beta E[S(\tilde{p})]
\end{aligned}

where we used the Nash Bargaining equation at the end. Now we can derive the job creation equation:

\begin{aligned}
\frac{1}{\delta} \frac{\theta}{m(1, \theta)} c(p) &=E[J(\tilde{p})] \\
&=(1-\beta) E[S(\tilde{p})] \\
&=(1-\beta) E[\tilde{p}-z+\delta(1-s) E[S(\tilde{p})]-\delta m(1, \tilde{\theta}) \beta E[S(\tilde{p})]\\
&=(1-\beta) E\left[\tilde{p}-z+\delta(1-s) \frac{1}{1-\beta} \frac{1}{\delta} \frac{\tilde{\theta}}{m(1, \tilde{\theta})} c(\tilde{p})-\delta m(1, \tilde{\theta}) \frac{\beta}{1-\beta} \frac{1}{\delta} \frac{\tilde{\theta}}{m(1, \tilde{\theta})} c(\tilde{p})\right] \\
&=E\left[(1-\beta)(\tilde{p}-z)+(1-s) \frac{\tilde{\theta}}{m(1, \tilde{\theta})} c(\tilde{p})-\beta \tilde{\theta} c(\tilde{p})\right]
\end{aligned}

and replacing by the functional form for $m(u,v)$ and $c(p)$ we have:
\begin{equation}\frac{1}{\delta}(1+\theta)^{\frac{1}{l}}\left(c_{k} p+c_{l} p^{\xi}\right)=E\left[(1-\beta)(\tilde{p}-z)+(1-s)(1+\tilde{\theta})^{\frac{1}{l}}\left(c_{k} \tilde{p}+c_{l} \tilde{p}^{\xi}\right)-\beta \tilde{\theta}\left(c_{k} \tilde{p}+c_{l} \tilde{p}^{\xi}\right)\right]\end{equation}


In [52]:
def jobcreation(θ):
    # theta (10,) for each state of P
    P       = np.exp(MC.state_values)  # (10,)
#     print(costvac(P))
    LHS     = 1 / δ * (1+θ) **(1/l) * costvac(P)  # (10,)
#     print(LHS)
    Future  = (1-β)*(P - z) + (1 - s)*(1+θ) **(1/l) * costvac(P) - β * θ * costvac(P)
#     print(Future)
    RHS     = MC.P @ Future
    return LHS - RHS


In [82]:
θ_0 = np.ones(10)
sol = optimize.root(jobcreation, θ_0)
θ   = sol.x
array = np.column_stack((np.arange(1,11,1), sol.x))

df = pd.DataFrame(array,
                   columns=['States', 'theta'])
HTML(df.round(4).to_html(index=False))                       

States,theta
1.0,1.3061
2.0,1.3179
3.0,1.3295
4.0,1.3408
5.0,1.3520
6.0,1.3630
7.0,1.3737
8.0,1.3843
9.0,1.3946
10.0,1.4048


## d) Computing the value functions

We first need to derive the wage setting condition in order to derive the value functions for a given p. We have:
\begin{equation}\begin{aligned}
J(p) &=p-w(p)+\delta((1-s) E[J(\tilde{p})]+s E[V(\tilde{p})]) \\
&=p-w(p)+\delta(1-s) E[J(\tilde{p})] \\
&=p-w(p)+\delta(1-s) \frac{1}{\delta} \frac{\theta}{m(1, \theta)} c(p) \\
&=p-w(p)+(1-s) \frac{\theta}{m(1, \theta)} c(p) \\
\Rightarrow(1-\beta) S(p) &=p-w(p)+(1-s) \frac{\theta}{m(1, \theta)} c(p)
\end{aligned}\end{equation}

and:

\begin{equation}\begin{aligned}
S(p) &=w(p)-z+\delta(1-s-m(1, \theta)) E[Q(\tilde{p})-U(\tilde{p})]+p-w(p)+\delta(1-s) E[J(\tilde{p})] \\
&=p-z+\delta(1-s-m(1, \theta)) \frac{\beta}{1-\beta} E[J(\tilde{p})]+\delta(1-s) E[J(\tilde{p})] \\
&=p-z+\left\{\delta(1-s-m(1, \theta)) \frac{1}{1-\beta}+\delta m(1, \theta)\right\} E[J(\tilde{p})] \\
&=p-z+\left\{\delta(1-s-m(1, \theta)) \frac{1}{1-\beta}+\delta m(1, \theta)\right\} \frac{1}{\delta} \frac{\theta}{m(1, \theta)} c(p) \\
&=p-z+(1-s-m(1, \theta) \beta) \frac{1}{1-\beta} \frac{\theta}{m(1, \theta)} c(p)
\end{aligned}\end{equation}

Substituting in the equation above we get:

\begin{equation}\begin{aligned}
w(p) &=\beta p+(1-\beta) z+\beta \theta c(p) \\
&=\beta p+(1-\beta) z+\beta \theta\left(c_{k} p+c_{l} p^{\varepsilon}\right)
\end{aligned}\end{equation}


In [84]:
def w(p, θ):
    return β*p + (1- β)*z + β*θ*costvac(p)

In [92]:
V = np.zeros(len(θ_0))

def J(p, θ):
    return p - w(p,θ) + (1 - s)*costvac(p)*(θ**l + 1)**(1/l)
    
def U(p, θ):
    return Q - (β/(1-β))*J(p,θ)

    

Computing $Q(p)$ is a little more involved. From the value function, we can replace $U(p) = Q(p) - (β/(1-β))J(p)$ to get:

\begin{equation}\begin{aligned}
Q(p) &=w(p)+\delta((1-s) E[Q(\tilde{p})]+s E[U(\tilde{p})]) \\
&=w(p)+\delta\left((1-s) E[Q(\tilde{p})]+s E\left[Q(\tilde{p})-\frac{\beta}{1-\beta} J(\tilde{p})\right]\right) \\
&=w(p)+\delta E[Q(\tilde{p})]-\delta s \frac{\beta}{1-\beta} \frac{1}{\delta} \frac{\theta}{m(1, \theta)} c(p) \\
&=w(p)+\delta E[Q(\tilde{p})]-s \frac{\beta}{1-\beta}\left(\theta^{l}+1\right)^{\frac{1}{l}}\left(c_{k} p+c_{l} p^{\varepsilon}\right)
\end{aligned}\end{equation}

In [86]:
def solveQ(Q):
    P       = np.exp(MC.state_values)
    LHS = Q # (10,1)
    EQ  = MC.P @ Q
    RHS = w(P, θ) + δ*EQ - s*(β/(1-β)) * (θ**l + 1)**(1/l) * costvac(P) 
    return LHS - RHS

In [87]:
Q_0 = np.ones(10)
sol = optimize.root(solveQ, Q_0)
Q   = sol.x


In [96]:
P = np.exp(MC.state_values)
array = np.column_stack((np.arange(1,11,1),θ, Q, U(P,θ), J(P, θ), V))

df2 = pd.DataFrame(array,
                   columns=['State', 'Tightness', 'Employed', 'Unemployed', 'Active', 'Vacant'])
HTML(df2.round(4).to_html(index=False))  

State,Tightness,Employed,Unemployed,Active,Vacant
1.0,1.3061,1114.2403,1111.9352,3.4576,0.0
2.0,1.3179,1115.3477,1112.9979,3.5246,0.0
3.0,1.3295,1116.4632,1114.0681,3.5926,0.0
4.0,1.3408,1117.5869,1115.1460,3.6614,0.0
5.0,1.3520,1118.7190,1116.2316,3.7311,0.0
6.0,1.3630,1119.8595,1117.3250,3.8018,0.0
7.0,1.3737,1121.0085,1118.4262,3.8735,0.0
8.0,1.3843,1122.1662,1119.5355,3.9461,0.0
9.0,1.3946,1123.3325,1120.6527,4.0197,0.0
10.0,1.4048,1124.5076,1121.7781,4.0943,0.0


In [88]:
Q

array([1114.24029749, 1115.3476544 , 1116.46316584, 1117.58692046,
       1118.71900782, 1119.85951842, 1121.00854365, 1122.16617588,
       1123.33250842, 1124.50763558])